In [ ]:
colab = 'google.colab' in str(get_ipython())

In [ ]:
# Setup for Colab only
if colab:
    !pip install git+https://github.com/finetuneanon/transformers@gpt-neo-localattention3
    !pip install git+https://github.com/corolla-johnson/mkultra.git#egg=mkultra --log PIP_LOG
    !pip install gdown
    !pip install datasets
    !pip install tqdm
    !nvidia-smi

In [ ]:
# If on Colab, mount your Google Drive first!
if colab:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
# Decide the length of your soft prompt in tokens.
# 20 to 100 is a good amount.
n_tokens = 100

# Specify the model directory or huggingface name.
if colab:
    model_dir = "/content/drive/MyDrive/models/gpt-neo-2.7B-halved/"
else:
    model_dir = "D:/Git Repos/mkultra/models/gpt-neo-2.7B-halved/" 

In [ ]:
from mkultra.tuning import GPTNeoPromptTuningLM
from transformers import GPT2TokenizerFast
from mkultra.soft_prompt import SoftPrompt
from transformers import Adafactor
import random
import torch
import math

In [ ]:
from mkultra.tuning import GPTNeoPromptTuningLM
from transformers import Adafactor
model = GPTNeoPromptTuningLM.from_pretrained(model_dir).half().to("cuda").train()
optimizer = Adafactor([model.get_soft_params()])

In [ ]:
for length in range(1, 2048, 5):
    input_ids = torch.ones(1, length).long().cuda().detach()

    try:
        torch.cuda.empty_cache()
        
        outputs = model(input_ids=input_ids, labels=input_ids, use_cache=True)
        loss = outputs.loss
        loss.backward()

        print(f"Block size {length-5} OK")
        del input_ids
    except RuntimeError:
        print(f"Maximum block size: {length-5}")
        break

del input_ids
torch.cuda.empty_cache()

if colab:
    !nvidia-smi